In [1]:
# !pip install pycapnp
# !pip install typer
# !pip install ipywidgets

In [2]:
# Packages
import asyncio
import zmq.asyncio
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import time

# Utils
from deepdrrzmq.utils import zmq_util, typer_util, server_util, config_util
from deepdrrzmq.utils.zmq_util import zmq_no_linger_context
from deepdrrzmq.utils.typer_util import unwrap_typer_param
from deepdrrzmq.utils.server_util import messages, make_response, DeepDRRServerException
from deepdrrzmq.utils.config_util import config, load_config

In [3]:
class ZMQAsyncMsgHandler:
    """
    Manage the setup and configuration of ZMQ publisher/subscriber sockets for communication over TCP. 
    It uses the asyncio variant of ZMQ for asynchronous operations.
    """
    def __init__(self):
        """
        Initializes the ZMQ connection handler with the address and port numbers for REP, PUB, and SUB sockets.

        :param addr: The address of the ZMQ server.
        :param rep_port: The port number for REP (reply) socket connections.
        :param pub_port: The port number for PUB (publish) socket connections.
        :param sub_port: The port number for SUB (subscribe) socket connections.
        :param hwm: The high water mark (HWM) for message buffering.
        """
        self.config = config
        # self.addr = self.config['network']['addr_localhost']
        self.addr = self.config['network']['addr_ip_eth']
        self.rep_port = self.config['network']['rep_port']
        self.pub_port = self.config['network']['pub_port']
        self.sub_port = self.config['network']['sub_port']
        self.hwm = self.config['network']['sub_port']
        self.context = zmq.asyncio.Context() # zmq.Context()
        self.setup()

    def setup(self):
        """
        Sets up ZMQ PUB and SUB sockets for communication.
        """
        # Setup the PUB socket
        self.pub_socket = self.context.socket(zmq.PUB)
        self.pub_socket.hwm = self.hwm
        self.pub_socket.connect(f"tcp://{self.addr}:{self.pub_port}")
        # Setup the SUB socket
        self.sub_socket = self.context.socket(zmq.SUB)
        self.sub_socket.hwm = self.hwm
        self.sub_socket.connect(f"tcp://{self.addr}:{self.sub_port}")
        self.sub_socket.subscribe(b"/loggerd/")


In [4]:
class LoggerControllerInterface:
    def __init__(self, ZMQAMH_instance):
        self.ZMQAMH_instance = ZMQAMH_instance
        self._create_widgets()
        self._bind_event_handlers()
        self._setup_layout()

    def _create_widgets(self):
        # logger widgets
        self.record_button = widgets.Button(description='New Session', disabled=True, icon='circle')
        self.stop_button = widgets.Button(description='Stop', disabled=True, icon='square')
        self.session_id = widgets.Text(value='None', description='Session ID:', disabled=True)
        self.record_status = widgets.Text(value='None', description='Recording Status:', disabled=True)
        # environment setting widgets
        self.update_setting = widgets.Button(description='Update Setting', disabled=False, icon='check', button_style='', tooltip='Click me')
        ## patient model widgets
        self.colortoggle = widgets.ToggleButtons(options=[('Opaque', 0), ('Transparent' ,1)], 
                                                 description='Patient Material:', disabled=False, button_style='')
        self.flippatient = widgets.ToggleButtons(options=[('Original',True), ('Flipped',False)], 
                                                 description='Flip Patient:', disabled=False, button_style='')
        ## view indicator widgets
        self.viewindicator = widgets.ToggleButtons(options=[('On',True), ('Off',False)], 
                                                   description='View Indicator ON/OFF:', disabled=False, button_style='')
        self.selfviewindicator = widgets.ToggleButtons(options=[('On',True), ('Off',False)],     
                                                       description='View Indicator Self-Select ON/OFF:', disabled=False, button_style='')
        self.viewindicatorselect = widgets.SelectMultiple(options=['Anteroposterior_View', 'Lateral_View', 'Inlet_View', 'Outlet_View', 'Obturator_Oblique_Left/Iliac_Oblique_Right', 'Obturator_Oblique_Right/Iliac_Oblique_Left', 'Obturator_Oblique_Inlet_Left/Iliac_Oblique_Inlet_Right','Obturator_Oblique_Outlet_Left/Iliac_Oblique_Outlet_Right','Obturator_Oblique_Inlet_Right/Iliac_Oblique_Inlet_Left','Obturator_Oblique_Outlet_Right/Iliac_Oblique_Outlet_Left', 'Teardrop_Left_View','Teardrop_Right_View','None'], 
                                                          value=['Lateral_View'], description='View Indicator Selection:', disabled=False)
        ## surgical instrument widgets
        self.kwireindicator = widgets.ToggleButtons(options=[('On',True), ('Off',False)], 
                                                    description='Kwire Error Indicator ON/OFF:', disabled=False, button_style='')
        self.kwireerrorselection = widgets.ToggleButtons(options=[('On',True), ('Off',False)], 
                                                         description='Kwire Error Indicator Self-Selection:', disabled=False, button_style='')
        self.corridorselection = widgets.Select(options=[('Left S1', 0), ('Left Ramus', 1), ('L Ramus Short', 2),('L Teardrop', 3),('Right S1', 4), ('Right Ramus', 5), ('Right Ramus Short', 6),('Right Teardrop', 7),('S 2', 8),('S 3', 9),('None',100)], 
                                                value=2, description='Corridor:')
        self.patientSwitch= widgets.Select(options=[('Previous', -1), ('Same', 0), ('Next', 1)],
                                        value=0,description='Patient Switch:',)

    def _bind_event_handlers(self):
        self.update_setting.on_click(lambda b: asyncio.create_task(self.updatesetting_clicked(b)))
        self.record_button.on_click(lambda b: asyncio.create_task(self.on_record_button_clicked(b)))
        self.record_button.on_click(lambda b: asyncio.create_task(self.updatesetting_clicked(b)))
        self.stop_button.on_click(lambda b: asyncio.create_task(self.on_stop_button_clicked(b)))
        
    def _setup_layout(self):
        # environment setting layout
        patient_model_layout = widgets.VBox([self.colortoggle, self.flippatient, self.patientSwitch])
        view_indicator_layout = widgets.VBox([self.viewindicator, self.selfviewindicator, self.viewindicatorselect])
        surgical_instrument_layout = widgets.VBox([self.kwireindicator, self.kwireerrorselection, self.corridorselection])
        environment_setting_layout = widgets.HBox([patient_model_layout, view_indicator_layout , surgical_instrument_layout, self.update_setting])
        display(environment_setting_layout)
        # logger layout
        logger_layout = widgets.HBox([self.record_button, self.stop_button, self.session_id, self.record_status])
        display(logger_layout)
    
    async def ui_update_loop(self):
        while True:
            print(f"enter while")
            try:
                print(f"enter try")
                topic, data = await asyncio.wait_for(
                    self.ZMQAMH_instance.sub_socket.recv_multipart(), 
                    timeout=3.0
                )
                
                if topic == b"/loggerd/status/":
                    print(f"enter loggerd/status/")
                    self.record_button.disabled = False
                    self.stop_button.disabled = False
                    with messages.LoggerStatus.from_bytes(data) as msg:
                        self.record_status.value = str(msg.recording)
                        self.session_id.value = msg.sessionId if msg.recording else "no session"
                        self.record_button.button_style = 'danger' if msg.recording else ''
            except asyncio.TimeoutError:
                print(f"enter asyncio timeout error")
                self.record_button.disabled = True
                self.stop_button.disabled = True
                self.session_id.value = "Disconnected"
                self.record_status.value = "Disconnected"
            except zmq.ZMQError as e:
                print(f"ZMQ Error: {e}")
            print(f"exit try except")

            await asyncio.sleep(0.1)

    async def on_record_button_clicked(self, b):
        await self.ZMQAMH_instance.pub_socket.send_multipart([b"/loggerd/start/", b""])
        print(f"Record button clicked, b = {b}")

    async def on_stop_button_clicked(self, b):
        await self.ZMQAMH_instance.pub_socket.send_multipart([b"/loggerd/stop/", b""])
        print(f"Stop button clicked, b = {b}")
        
    async def updatesetting_clicked(self, b):
        response_topic = "/mp/setting/webgui/SettingManager/"
        msg = messages.SyncedSetting.new_message()
        msg.timestamp = msg.timestamp = float(np.float64(time.time()).item())
        msg.clientId = 'webgui'
        uiControlVar = msg.setting.init("uiControl")
        uiControlVar.patientMaterial = self.colortoggle.value
        uiControlVar.flippatient = self.flippatient.value    
        uiControlVar.carmIndicator = self.viewindicator.value
        uiControlVar.viewIndicatorselfselect = self.selfviewindicator.value
        uiControlVar.annotationError = self.viewindicatorselect.value
        uiControlVar.corridorIndicator = self.kwireindicator.value
        uiControlVar.webcorridorerrorselect = self.kwireerrorselection.value
        uiControlVar.webcorridorselection = self.corridorselection.value
        uiControlVar.patientSwitchParam = self.patientSwitch.value
        await self.ZMQAMH_instance.pub_socket.send_multipart([response_topic.encode(), msg.to_bytes()])
        print(f"Update Setting button clicked, b = {b}")


In [5]:
# ZMQAMH = ZMQAsyncMsgHandler()
# LCI = LoggerControllerInterface(ZMQAMH)

# try:
#     ui_update_task.cancel()
# except:
#     pass

# ui_update_task = asyncio.create_task(LCI.ui_update_loop())

async def main():
    ZMQAMH = ZMQAsyncMsgHandler()
    LCI = LoggerControllerInterface(ZMQAMH)

    try:
        ui_update_task.cancel()
    except NameError:
        pass

    ui_update_task = asyncio.create_task(LCI.ui_update_loop())

if __name__ == "__main__":
    await main()

enter while
enter try


enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
enter while
enter try
enter loggerd/status/
exit try except
en